# Exploratory Data Analysis
Quality control and visualization of RNA-seq data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load count data
counts = pd.read_csv('../results/counts/gene_counts.csv', index_col=0)
print(f"Loaded {counts.shape[0]} genes x {counts.shape[1]} samples")
counts.head()

In [ ]:
# Basic statistics
print("Total counts per sample:")
print(counts.sum())
print("\nGenes with zero counts:", (counts.sum(axis=1) == 0).sum())

In [ ]:
# Filter low-count genes
min_counts = 10
min_samples = 2
keep = (counts >= min_counts).sum(axis=1) >= min_samples
counts_filtered = counts[keep]
print(f"Genes after filtering: {counts_filtered.shape[0]}")

In [ ]:
# CPM normalization
cpm = counts_filtered.div(counts_filtered.sum()) * 1e6
log_cpm = np.log2(cpm + 1)

In [ ]:
# Distribution of gene expression
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Raw counts distribution
for col in counts_filtered.columns:
    axes[0].hist(np.log2(counts_filtered[col] + 1), bins=50, alpha=0.5, label=col)
axes[0].set_xlabel('log2(counts + 1)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Raw Counts Distribution')
axes[0].legend()

# CPM distribution
for col in log_cpm.columns:
    axes[1].hist(log_cpm[col], bins=50, alpha=0.5, label=col)
axes[1].set_xlabel('log2(CPM + 1)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Normalized (CPM) Distribution')
axes[1].legend()

plt.tight_layout()
plt.savefig('../results/figures/expression_distribution.pdf')
plt.show()

In [ ]:
# Sample correlation heatmap
corr = log_cpm.corr(method='pearson')

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='RdBu_r', vmin=0.8, vmax=1, 
            square=True, fmt='.3f')
plt.title('Sample Correlation (Pearson)')
plt.tight_layout()
plt.savefig('../results/figures/sample_correlation.pdf')
plt.show()

In [ ]:
# PCA analysis
scaler = StandardScaler()
scaled_data = scaler.fit_transform(log_cpm.T)

pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

# Plot PCA
plt.figure(figsize=(8, 6))
colors = ['blue', 'red', 'red']  # Control, Treatment, Treatment
for i, sample in enumerate(log_cpm.columns):
    plt.scatter(pca_result[i, 0], pca_result[i, 1], 
                c=colors[i], s=100, label=sample)
    plt.annotate(sample, (pca_result[i, 0], pca_result[i, 1]), 
                 xytext=(5, 5), textcoords='offset points')

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
plt.title('PCA of Gene Expression')
plt.legend()
plt.tight_layout()
plt.savefig('../results/figures/pca_sklearn.pdf')
plt.show()

In [ ]:
# Top variable genes
gene_variance = log_cpm.var(axis=1).sort_values(ascending=False)
top_genes = gene_variance.head(100).index

# Heatmap of top variable genes
plt.figure(figsize=(10, 12))
sns.clustermap(log_cpm.loc[top_genes], 
               cmap='RdBu_r', 
               z_score=0,
               figsize=(8, 10))
plt.savefig('../results/figures/top_genes_heatmap.pdf')
plt.show()

In [ ]:
print("Exploratory analysis complete!")